In [6]:
from gdeep.topactivation import TopactivationFC as TFC
from gdeep.pipeline import Pipeline
from torch.utils.tensorboard import SummaryWriter

from gdeep.models import FFNet
from torch import nn
import torch
from gdeep.data import TorchDataLoader

!pip3 install xitorch
from gdeep.models import ModelExtractor
from gdeep.topactivation.spectral_analysisTorch import LaplacianOperator
import matplotlib.pyplot as plt 

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU!")
else:
    DEVICE = torch.device("cpu")
    
writer = SummaryWriter()
dl = TorchDataLoader(name="MNIST")
dl_tr, dl_ts = dl.build_dataloaders(batch_size=32)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Using GPU!


In [80]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()        
        conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=8,            
                kernel_size=3,              
                stride=1,                   
                padding=2,bias=False                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        
        conv2 = nn.Sequential(         
            nn.Conv2d(8, 16, 3, 1, 2,bias=False),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )        # fully connected layer, output 10 classes
        
        conv3 = nn.Sequential(         
            nn.Conv2d(16, 32, 3, 1, 2,bias=False),     
            nn.ReLU(),                      
            nn.MaxPool2d(7),                
        )        #
        
        out = nn.Sequential(nn.Flatten(),nn.Linear(32 , 10,bias=False))
        
        
        
        self.layers=nn.Sequential(conv1,conv2,conv3,out)
        
    def forward(self, x):
        x = self.layers(x)   # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        return x   # return x for visualization
    
    
    
model = CNN()
loss_fn = nn.CrossEntropyLoss()
pipe = Pipeline(model, (dl_tr, dl_ts), loss_fn, writer)

In [85]:
#List layers : 
layers = [module for module in model.modules() if isinstance(module, nn.Sequential)]
print(len(layers))

5


In [22]:


for x in dl_tr:
    print(model(x[0]).shape)
    break 

torch.Size([32, 10])


## Train the CNN

In [82]:
optimizer = torch.optim.SGD
pipe.train(optimizer, 5, False, {"lr": 0.1}, n_accumulated_grads=5)

Epoch 1
-------------------------------
Epoch training loss: 1.557888 	Epoch training accuracy: 45.62%                                                                
Time taken for this epoch: 10.00s
Learning rate value: 0.10000000
Validation results: 
 Accuracy: 89.06%,                 Avg loss: 0.377016 

Epoch 2
-------------------------------
Epoch training loss: 0.279159 	Epoch training accuracy: 91.45%                                                                
Time taken for this epoch: 10.00s
Learning rate value: 0.10000000
Validation results: 
 Accuracy: 92.75%,                 Avg loss: 0.235724 

Epoch 3
-------------------------------
Epoch training loss: 0.174080 	Epoch training accuracy: 94.63%                                      500 ]                     
Time taken for this epoch: 10.00s
Learning rate value: 0.10000000
Validation results: 
 Accuracy: 95.01%,                 Avg loss: 0.162477 

Epoch 4
-------------------------------
Epoch training loss: 0.129600 

(0.11948767622249822, 96.29166666666666)